In [ ]:
"""
This notebook truncates BISC recordings (LMP and Spectrograms) in time domain (to match the
good behaving segments) and in frequency domain (remove bins with overlapping contents)
"""

In [ ]:
%load_ext autoreload
%autoreload 2
import os, sys

import numpy as np

from matplotlib import pyplot as plt
from matplotlib_settings import set_plot_settings, reset_plot_settings

# Set the plot settings
set_plot_settings()

# import global variables
from utils_motor_global import *

ROOT_SAVE_DIR = f'{REC_DIR}/6_truncate'
ROOT_LOAD_DIR = f'{REC_DIR}/5_LMP_MT'

In [ ]:
for session in GOOD_SESSIONS:
    keys = [key for key in SESSION_KEYS if key.startswith(f'{session:003}')]

    for key in keys:
        load_dir = f'{ROOT_LOAD_DIR}/{key}'
        if not os.path.exists(load_dir):
            continue
        print(f'processing {key}...')

        """ save directory """
        save_data_dir = f'{ROOT_SAVE_DIR}/{key}'
        if not os.path.exists(save_data_dir):
            os.makedirs(save_data_dir)

        """ load recording data """
        spect_t     = np.load(f'{load_dir}/spect_t_{key}.npy')
        spect_f     = np.load(f'{load_dir}/spect_f_{key}.npy')
        beta_fidxs  = np.load(f'{load_dir}/beta_fidxs_{key}.npy')
        lga_fidxs   = np.load(f'{load_dir}/lga_fidxs_{key}.npy')
        hga_fidxs   = np.load(f'{load_dir}/hga_fidxs_{key}.npy')

        good_chs    = np.load(f'{load_dir}/good_channels_{key}.npy')

        spect       = np.load(f'{load_dir}/spect_{key}.npy')
        lmp_data    = np.load(f'{load_dir}/lmp_{key}.npy')

        """ truncate unnecessary frequency bins """
        # since I am partitioning frequency, discard the overlapping bins
        m = round(2*W_MT_SCALO/spect_f[1])
        assert m == 4
        spect = spect[:,:,::m]
        spect_f = spect_f[::m]
        beta_fidxs = beta_fidxs//m
        lga_fidxs  = lga_fidxs//m
        hga_fidxs  = hga_fidxs//m

        # also discard bins greater than 200 Hz
        spect = spect[:,:,spect_f <= SCALO_MAX_FREQ]
        spect_f = spect_f[spect_f <= SCALO_MAX_FREQ]

        """ load motion data """
        motion_dir = f'{MOTION_DIR}/{session:03}'
        motion_t  = np.load(f'{motion_dir}/pos_t_session_{key}.npy')

        """ truncate unncessary time points from BISC recording """
        # Truncate spectrogram
        t0, mt0 = spect_t[0], motion_t[0] + FULL_TAU_START
        t1, mt1 = spect_t[-1], motion_t[-1] + FULL_TAU_END
        if t0 < mt0: # truncate time points before good behaving segment
            t_idx0 = np.where(spect_t > mt0)[0][0]
        else:
            t_idx0 = 0

        if t1 > mt1: # truncate time points after good behaving segment
            t_idx1 = np.where(spect_t < mt1)[0][-1] + 1
        else:
            t_idx1 = len(spect_t) + 1

        spect_t = spect_t[t_idx0:t_idx1]
        spect = spect[:,t_idx0:t_idx1,:]
        lmp_data = lmp_data[:,t_idx0:t_idx1]

        # make sure motion_t range covers scalogram range
        assert motion_t[0] < spect_t[0] - FULL_TAU_START
        assert motion_t[-1] > spect_t[-1] - FULL_TAU_END

        """ save data """
        # carry overs
        np.save(f'{save_data_dir}/good_channels_{key}.npy', good_chs)

        # truncated spectrogram and LMP
        np.save(f'{save_data_dir}/spect_{key}.npy', spect)
        np.save(f'{save_data_dir}/lmp_{key}.npy', lmp_data)
        np.save(f'{save_data_dir}/spect_t_{key}.npy', spect_t) 
        np.save(f'{save_data_dir}/spect_f_{key}.npy', spect_f) 
        np.save(f'{save_data_dir}/beta_fidxs_{key}.npy', beta_fidxs)
        np.save(f'{save_data_dir}/lga_fidxs_{key}.npy', lga_fidxs)
        np.save(f'{save_data_dir}/hga_fidxs_{key}.npy', hga_fidxs)

In [ ]:
# for key in SESSION_KEYS:
#     # fn = f'beta_fidxs_{key}.npy'
#     # fn = f'lga_fidxs_{key}.npy'
#     # fn = f'hga_fidxs_{key}.npy'

#     # fn = f'good_channels_{key}.npy'
#     # fn = f'lmp_{key}.npy'
#     fn = f'spect_{key}.npy'
#     # fn = f'spect_t_{key}.npy'
#     # fn = f'spect_f_{key}.npy'

#     dir0 = f'./combined_preprocessed_v3/1_align_and_chop_again/{key}'
#     dir1 = f'./recording_preprocessed_v3/6_truncate/{key}'

#     X0 = np.load(f'{dir0}/{fn}')
#     X1 = np.load(f'{dir1}/{fn}')
#     assert np.allclose(X0, X1, equal_nan=True)